In [1]:
import json
import os
import requests
from typing import List, Dict, Any
import sys

CFBD_API_URL = "https://api.collegefootballdata.com/games"
try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # __file__ is not defined (e.g., running in an interactive shell or notebook)
    BASE_DIR = os.getcwd()

CONFIG_PATH = os.path.join(BASE_DIR, "config.json")


def load_config(path: str = CONFIG_PATH) -> Dict[str, Any]:
    """
    Load configuration from a JSON file.

    Expected format (at minimum):

    {
      "CFBD_API_KEY": "your_real_key_here"
    }
    """
    try:
        with open(path, "r", encoding="utf-8") as f:
            cfg = json.load(f)
    except FileNotFoundError:
        raise RuntimeError(f"Config file not found at '{path}'. "
                           f"Create it with your CFBD_API_KEY.")
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Error parsing JSON in config file '{path}': {e}")
    return cfg


config = load_config()
CFBD_API_KEY = config.get("CFBD_API_KEY")


if CFBD_API_KEY is None:
    raise RuntimeError("Set CFBD_API_KEY env var with your CollegeFootballData API key.")


# -------------------------------------------------------------------
# 1. You plug in your real SAMPLE_TEAMS list here (BYU must be team_id 1)
# -------------------------------------------------------------------

SAMPLE_TEAMS = [
    {"team_id": 1, "team_name": "BYU", "mascot": "Cougars", "conference": "Big 12"},
    {"team_id": 2, "team_name": "Air Force", "mascot": "Falcons", "conference": "Mountain West"},
    {"team_id": 3, "team_name": "Akron", "mascot": "Zips", "conference": "MAC"},
    {"team_id": 4, "team_name": "Alabama", "mascot": "Crimson Tide", "conference": "SEC"},
    {"team_id": 5, "team_name": "Appalachian State", "mascot": "Mountaineers", "conference": "Sun Belt"},
    {"team_id": 6, "team_name": "Arizona", "mascot": "Wildcats", "conference": "Big 12"},
    {"team_id": 7, "team_name": "Arizona State", "mascot": "Sun Devils", "conference": "Big 12"},
    {"team_id": 8, "team_name": "Arkansas", "mascot": "Razorbacks", "conference": "SEC"},
    {"team_id": 9, "team_name": "Arkansas State", "mascot": "Red Wolves", "conference": "Sun Belt"},
    {"team_id": 10, "team_name": "Army", "mascot": "Black Knights", "conference": "American[d]"},
    {"team_id": 11, "team_name": "Auburn", "mascot": "Tigers", "conference": "SEC"},
    {"team_id": 12, "team_name": "Ball State", "mascot": "Cardinals", "conference": "MAC"},
    {"team_id": 13, "team_name": "Baylor", "mascot": "Bears", "conference": "Big 12"},
    {"team_id": 14, "team_name": "Boise State", "mascot": "Broncos", "conference": "Mountain West[e]"},
    {"team_id": 15, "team_name": "Boston College", "mascot": "Eagles", "conference": "ACC"},
    {"team_id": 16, "team_name": "Bowling Green", "mascot": "Falcons", "conference": "MAC"},
    {"team_id": 17, "team_name": "Buffalo", "mascot": "Bulls", "conference": "MAC"},
    {"team_id": 18, "team_name": "California", "mascot": "Golden Bears", "conference": "ACC"},
    {"team_id": 19, "team_name": "Central Michigan", "mascot": "Chippewas", "conference": "MAC"},
    {"team_id": 20, "team_name": "Charlotte", "mascot": "49ers", "conference": "American"},
    {"team_id": 21, "team_name": "Cincinnati", "mascot": "Bearcats", "conference": "Big 12"},
    {"team_id": 22, "team_name": "Clemson", "mascot": "Tigers", "conference": "ACC"},
    {"team_id": 23, "team_name": "Coastal Carolina", "mascot": "Chanticleers", "conference": "Sun Belt"},
    {"team_id": 24, "team_name": "Colorado", "mascot": "Buffaloes", "conference": "Big 12"},
    {"team_id": 25, "team_name": "Colorado State", "mascot": "Rams", "conference": "Mountain West[e]"},
    {"team_id": 26, "team_name": "Delaware", "mascot": "Fightin' Blue Hens", "conference": "CUSA"},
    {"team_id": 27, "team_name": "Duke", "mascot": "Blue Devils", "conference": "ACC"},
    {"team_id": 28, "team_name": "East Carolina", "mascot": "Pirates", "conference": "American"},
    {"team_id": 29, "team_name": "Eastern Michigan", "mascot": "Eagles", "conference": "MAC"},
    {"team_id": 30, "team_name": "FIU", "mascot": "Panthers", "conference": "CUSA"},
    {"team_id": 31, "team_name": "Florida", "mascot": "Gators", "conference": "SEC"},
    {"team_id": 32, "team_name": "Florida Atlantic", "mascot": "Owls", "conference": "American"},
    {"team_id": 33, "team_name": "Florida State", "mascot": "Seminoles", "conference": "ACC"},
    {"team_id": 34, "team_name": "Fresno State", "mascot": "Bulldogs", "conference": "Mountain West[e]"},
    {"team_id": 35, "team_name": "Georgia", "mascot": "Bulldogs", "conference": "SEC"},
    {"team_id": 36, "team_name": "Georgia Southern", "mascot": "Eagles", "conference": "Sun Belt"},
    {"team_id": 37, "team_name": "Georgia State", "mascot": "Panthers", "conference": "Sun Belt"},
    {"team_id": 38, "team_name": "Georgia Tech", "mascot": "Yellow Jackets", "conference": "ACC"},
    {"team_id": 39, "team_name": "Hawaiʻi", "mascot": "Rainbow Warriors", "conference": "Mountain West[k]"},
    {"team_id": 40, "team_name": "Houston", "mascot": "Cougars", "conference": "Big 12"},
    {"team_id": 41, "team_name": "Illinois", "mascot": "Fighting Illini", "conference": "Big Ten"},
    {"team_id": 42, "team_name": "Indiana", "mascot": "Hoosiers", "conference": "Big Ten"},
    {"team_id": 43, "team_name": "Iowa", "mascot": "Hawkeyes", "conference": "Big Ten"},
    {"team_id": 44, "team_name": "Iowa State", "mascot": "Cyclones", "conference": "Big 12"},
    {"team_id": 45, "team_name": "Jacksonville State", "mascot": "Gamecocks", "conference": "CUSA"},
    {"team_id": 46, "team_name": "James Madison", "mascot": "Dukes", "conference": "Sun Belt"},
    {"team_id": 47, "team_name": "Kansas", "mascot": "Jayhawks", "conference": "Big 12"},
    {"team_id": 48, "team_name": "Kansas State", "mascot": "Wildcats", "conference": "Big 12"},
    {"team_id": 49, "team_name": "Kennesaw State", "mascot": "Owls", "conference": "CUSA"},
    {"team_id": 50, "team_name": "Kent State", "mascot": "Golden Flashes", "conference": "MAC"},
    {"team_id": 51, "team_name": "Kentucky", "mascot": "Wildcats", "conference": "SEC"},
    {"team_id": 52, "team_name": "LSU", "mascot": "Tigers", "conference": "SEC"},
    {"team_id": 53, "team_name": "Liberty", "mascot": "Flames", "conference": "CUSA"},
    {"team_id": 54, "team_name": "Louisiana", "mascot": "Ragin' Cajuns", "conference": "Sun Belt"},
    {"team_id": 55, "team_name": "Louisiana Tech", "mascot": "Bulldogs", "conference": "CUSA"},
    {"team_id": 56, "team_name": "Louisville", "mascot": "Cardinals", "conference": "ACC"},
    {"team_id": 57, "team_name": "Marshall", "mascot": "Thundering Herd", "conference": "Sun Belt"},
    {"team_id": 58, "team_name": "Maryland", "mascot": "Terrapins", "conference": "Big Ten"},
    {"team_id": 59, "team_name": "Memphis", "mascot": "Tigers", "conference": "American"},
    {"team_id": 60, "team_name": "Miami (FL)", "mascot": "Hurricanes", "conference": "ACC"},
    {"team_id": 61, "team_name": "Miami (OH)", "mascot": "RedHawks", "conference": "MAC"},
    {"team_id": 62, "team_name": "Michigan", "mascot": "Wolverines", "conference": "Big Ten"},
    {"team_id": 63, "team_name": "Michigan State", "mascot": "Spartans", "conference": "Big Ten"},
    {"team_id": 64, "team_name": "Middle Tennessee", "mascot": "Blue Raiders", "conference": "CUSA"},
    {"team_id": 65, "team_name": "Minnesota", "mascot": "Golden Gophers", "conference": "Big Ten"},
    {"team_id": 66, "team_name": "Mississippi State", "mascot": "Bulldogs", "conference": "SEC"},
    {"team_id": 67, "team_name": "Missouri", "mascot": "Tigers", "conference": "SEC"},
    {"team_id": 68, "team_name": "Missouri State", "mascot": "Bears", "conference": "CUSA"},
    {"team_id": 69, "team_name": "NC State", "mascot": "Wolfpack", "conference": "ACC"},
    {"team_id": 70, "team_name": "Navy", "mascot": "Midshipmen", "conference": "American[d]"},
    {"team_id": 71, "team_name": "Nebraska", "mascot": "Cornhuskers", "conference": "Big Ten"},
    {"team_id": 72, "team_name": "Nevada", "mascot": "Wolf Pack", "conference": "Mountain West"},
    {"team_id": 73, "team_name": "New Mexico", "mascot": "Lobos", "conference": "Mountain West"},
    {"team_id": 74, "team_name": "New Mexico State", "mascot": "Aggies", "conference": "CUSA"},
    {"team_id": 75, "team_name": "North Carolina", "mascot": "Tar Heels", "conference": "ACC"},
    {"team_id": 76, "team_name": "North Texas", "mascot": "Mean Green", "conference": "American"},
    {"team_id": 77, "team_name": "Northern Illinois", "mascot": "Huskies", "conference": "MAC"},
    {"team_id": 78, "team_name": "Northwestern", "mascot": "Wildcats", "conference": "Big Ten"},
    {"team_id": 79, "team_name": "Notre Dame", "mascot": "Fighting Irish", "conference": "Independent"},
    {"team_id": 80, "team_name": "Ohio", "mascot": "Bobcats", "conference": "MAC"},
    {"team_id": 81, "team_name": "Ohio State", "mascot": "Buckeyes", "conference": "Big Ten"},
    {"team_id": 82, "team_name": "Oklahoma", "mascot": "Sooners", "conference": "SEC"},
    {"team_id": 83, "team_name": "Oklahoma State", "mascot": "Cowboys", "conference": "Big 12"},
    {"team_id": 84, "team_name": "Old Dominion", "mascot": "Monarchs", "conference": "Sun Belt"},
    {"team_id": 85, "team_name": "Ole Miss", "mascot": "Rebels", "conference": "SEC"},
    {"team_id": 86, "team_name": "Oregon", "mascot": "Ducks", "conference": "Big Ten"},
    {"team_id": 87, "team_name": "Oregon State", "mascot": "Beavers", "conference": "Pac-12"},
    {"team_id": 88, "team_name": "Penn State", "mascot": "Nittany Lions", "conference": "Big Ten"},
    {"team_id": 89, "team_name": "Pittsburgh", "mascot": "Panthers", "conference": "ACC"},
    {"team_id": 90, "team_name": "Purdue", "mascot": "Boilermakers", "conference": "Big Ten"},
    {"team_id": 91, "team_name": "Rice", "mascot": "Owls", "conference": "American"},
    {"team_id": 92, "team_name": "Rutgers", "mascot": "Scarlet Knights", "conference": "Big Ten"},
    {"team_id": 93, "team_name": "SMU", "mascot": "Mustangs", "conference": "ACC"},
    {"team_id": 94, "team_name": "Sam Houston", "mascot": "Bearkats", "conference": "CUSA"},
    {"team_id": 95, "team_name": "San Diego State", "mascot": "Aztecs", "conference": "Mountain West[e]"},
    {"team_id": 96, "team_name": "San Jose State", "mascot": "Spartans", "conference": "Mountain West"},
    {"team_id": 97, "team_name": "South Alabama", "mascot": "Jaguars", "conference": "Sun Belt"},
    {"team_id": 98, "team_name": "South Carolina", "mascot": "Gamecocks", "conference": "SEC"},
    {"team_id": 99, "team_name": "South Florida", "mascot": "Bulls", "conference": "American"},
    {"team_id": 100, "team_name": "Southern Miss", "mascot": "Golden Eagles", "conference": "Sun Belt"},
    {"team_id": 101, "team_name": "Stanford", "mascot": "Cardinal", "conference": "ACC"},
    {"team_id": 102, "team_name": "Syracuse", "mascot": "Orange", "conference": "ACC"},
    {"team_id": 103, "team_name": "TCU", "mascot": "Horned Frogs", "conference": "Big 12"},
    {"team_id": 104, "team_name": "Temple", "mascot": "Owls", "conference": "American"},
    {"team_id": 105, "team_name": "Tennessee", "mascot": "Volunteers", "conference": "SEC"},
    {"team_id": 106, "team_name": "Texas", "mascot": "Longhorns", "conference": "SEC"},
    {"team_id": 107, "team_name": "Texas A&M", "mascot": "Aggies", "conference": "SEC"},
    {"team_id": 108, "team_name": "Texas State", "mascot": "Bobcats", "conference": "Sun Belt"},
    {"team_id": 109, "team_name": "Texas Tech", "mascot": "Red Raiders", "conference": "Big 12"},
    {"team_id": 110, "team_name": "Toledo", "mascot": "Rockets", "conference": "MAC"},
    {"team_id": 111, "team_name": "Troy", "mascot": "Trojans", "conference": "Sun Belt"},
    {"team_id": 112, "team_name": "Tulane", "mascot": "Green Wave", "conference": "American"},
    {"team_id": 113, "team_name": "Tulsa", "mascot": "Golden Hurricane", "conference": "American"},
    {"team_id": 114, "team_name": "UAB", "mascot": "Blazers", "conference": "American"},
    {"team_id": 115, "team_name": "UCF", "mascot": "Knights", "conference": "Big 12"},
    {"team_id": 116, "team_name": "UCLA", "mascot": "Bruins", "conference": "Big Ten"},
    {"team_id": 117, "team_name": "UConn", "mascot": "Huskies", "conference": "Independent"},
    {"team_id": 118, "team_name": "ULM", "mascot": "Warhawks", "conference": "Sun Belt"},
    {"team_id": 119, "team_name": "UMass", "mascot": "Minutemen", "conference": "MAC"},
    {"team_id": 120, "team_name": "UNLV", "mascot": "Rebels", "conference": "Mountain West"},
    {"team_id": 121, "team_name": "USC", "mascot": "Trojans", "conference": "Big Ten"},
    {"team_id": 122, "team_name": "UTEP", "mascot": "Miners", "conference": "CUSA[ag]"},
    {"team_id": 123, "team_name": "UTSA", "mascot": "Roadrunners", "conference": "American"},
    {"team_id": 124, "team_name": "Utah", "mascot": "Utes", "conference": "Big 12"},
    {"team_id": 125, "team_name": "Utah State", "mascot": "Aggies", "conference": "Mountain West[e]"},
    {"team_id": 126, "team_name": "Vanderbilt", "mascot": "Commodores", "conference": "SEC"},
    {"team_id": 127, "team_name": "Virginia", "mascot": "Cavaliers", "conference": "ACC"},
    {"team_id": 128, "team_name": "Virginia Tech", "mascot": "Hokies", "conference": "ACC"},
    {"team_id": 129, "team_name": "Wake Forest", "mascot": "Demon Deacons", "conference": "ACC"},
    {"team_id": 130, "team_name": "Washington", "mascot": "Huskies", "conference": "Big Ten"},
    {"team_id": 131, "team_name": "Washington State", "mascot": "Cougars", "conference": "Pac-12"},
    {"team_id": 132, "team_name": "West Virginia", "mascot": "Mountaineers", "conference": "Big 12"},
    {"team_id": 133, "team_name": "Western Kentucky", "mascot": "Hilltoppers", "conference": "CUSA"},
    {"team_id": 134, "team_name": "Western Michigan", "mascot": "Broncos", "conference": "MAC"},
    {"team_id": 135, "team_name": "Wisconsin", "mascot": "Badgers", "conference": "Big Ten"},
    {"team_id": 136, "team_name": "Wyoming", "mascot": "Cowboys", "conference": "Mountain West"},
]


# Build quick lookup from team name to team_id
TEAM_NAME_TO_ID = {t["team_name"]: t["team_id"] for t in SAMPLE_TEAMS}


def fetch_2025_fbs_games() -> List[Dict[str, Any]]:
    """
    Call CFBD to get all completed 2025 FBS regular-season games.
    """
    headers = {
        "Authorization": f"Bearer {CFBD_API_KEY}",
    }
    params = {
        "year": 2025,
        "seasonType": "regular",
        "division": "fbs",
    }
    resp = requests.get(CFBD_API_URL, headers=headers, params=params, timeout=60)
    
    resp.raise_for_status()
    games = resp.json()
    
    # Filter out games without final scores (if any)
    games = [g for g in games if g.get("homePoints") is not None and g.get("awayPoints") is not None]
    
    return games


def map_team_name_to_id(team_name: str) -> int:
    """
    Map CFBD team name to your team_id.
    If you have slightly different naming, add manual overrides here.
    """
    # Direct match first
    if team_name in TEAM_NAME_TO_ID:
        return TEAM_NAME_TO_ID[team_name]

    # Example manual tweaks (add as needed):
    name_fixes = {
        "Miami (FL)": "Miami",    # if you chose different canonical names
        # "UTSA": "UTSA",
    }
    if team_name in name_fixes and name_fixes[team_name] in TEAM_NAME_TO_ID:
        return TEAM_NAME_TO_ID[name_fixes[team_name]]

    raise KeyError(f"No team_id found for team_name '{team_name}'. "
                   f"Add it to SAMPLE_TEAMS or a name_fixes mapping.")


def build_sample_games(cfbd_games: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Convert CFBD game objects into your SAMPLE_GAMES format:

    {
        "game_id": int,
        "date": "YYYY-MM-DD",
        "home_team_id": int,
        "away_team_id": int,
        "home_score": int,
        "away_score": int,
        "neutral_site": 0/1,
    }
    """
    sample_games = []
    # Sort by kickoff date so game_id roughly follows time
    cfbd_games_sorted = sorted(cfbd_games, key=lambda g: g["startDate"])

    for i, g in enumerate(cfbd_games_sorted, start=1):
        home_team = g["homeTeam"]
        away_team = g["awayTeam"]
        
        try:
            home_id = map_team_name_to_id(home_team)
            away_id = map_team_name_to_id(away_team)
        except KeyError as e:
            # If you don't care about a few unmatched games, you can skip them here
            print(f"Skipping game due to unmatched team name: {e}")
            continue

        # CFBD uses ISO datetimes in start_date, e.g. "2025-08-30T17:00:00.000Z"
        date_str = g["startDate"][:10]  # "YYYY-MM-DD"

        neutral_flag = 1 if g.get("neutral_site") else 0

        sample_games.append({
            "game_id": i,
            "date": date_str,
            "home_team_id": home_id,
            "away_team_id": away_id,
            "home_score": int(g["homePoints"]),
            "away_score": int(g["awayPoints"]),
            "neutral_site": neutral_flag,
        })

    return sample_games


def format_as_python_list(sample_games: List[Dict[str, Any]]) -> str:
    """
    Dump SAMPLE_GAMES as Python code.
    """
    lines = ["SAMPLE_GAMES = ["]
    for g in sample_games:
        line = (
            f'    {{"game_id": {g["game_id"]}, '
            f'"date": "{g["date"]}", '
            f'"home_team_id": {g["home_team_id"]}, '
            f'"away_team_id": {g["away_team_id"]}, '
            f'"home_score": {g["home_score"]}, '
            f'"away_score": {g["away_score"]}, '
            f'"neutral_site": {g["neutral_site"]}}},'
        )
        lines.append(line)
    lines.append("]")
    return "\n".join(lines)


def main():
    games = fetch_2025_fbs_games()
    
    sample_games = build_sample_games(games)
    print(f"# Fetched {len(sample_games)} completed FBS games for 2025\n")
    print(format_as_python_list(sample_games))


if __name__ == "__main__":
    main()


Skipping game due to unmatched team name: "No team_id found for team_name 'Nicholls'. Add it to SAMPLE_TEAMS or a name_fixes mapping."
Skipping game due to unmatched team name: "No team_id found for team_name 'Idaho State'. Add it to SAMPLE_TEAMS or a name_fixes mapping."
Skipping game due to unmatched team name: "No team_id found for team_name 'Portland State'. Add it to SAMPLE_TEAMS or a name_fixes mapping."
Skipping game due to unmatched team name: "No team_id found for team_name 'Southern'. Add it to SAMPLE_TEAMS or a name_fixes mapping."
Skipping game due to unmatched team name: "No team_id found for team_name 'Hawai'i'. Add it to SAMPLE_TEAMS or a name_fixes mapping."
Skipping game due to unmatched team name: "No team_id found for team_name 'Central Washington'. Add it to SAMPLE_TEAMS or a name_fixes mapping."
Skipping game due to unmatched team name: "No team_id found for team_name 'Lafayette'. Add it to SAMPLE_TEAMS or a name_fixes mapping."
Skipping game due to unmatched team 